<a href="https://colab.research.google.com/github/UiinKim/UiinKim/blob/main/%EC%9B%8C%EB%93%9C_%EC%9E%84%EB%B2%A0%EB%94%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

In [2]:
#데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename='ted_en-20160408.xml')

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x79040c3afe80>)

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [20]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF-8')
target_text=etree.parse(targetXML) #Xml파일 간단하게 파싱

# xml 파일로부터 <content>와 </content>사이의 내용만 가져온다.
parse_text='\n'.join(target_text.xpath('//content/text()'))

#정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거
content_text = re.sub(r'\([^)]*\)', '', parse_text) #괄호로 구성된 내용을 제거

#입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행
sent_text=sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환
normalized_text=[]
for string in sent_text:
  tokens=re.sub(r"[^a-z0-9]+"," ",string.lower()) #sentence.lower() 상태에서 a-z0-9가 아닌 것들은 모두 ' '으로 바꾼다.
  normalized_text.append(tokens)

result=[word_tokenize(sentence) for sentence in normalized_text]

In [21]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [22]:
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


In [23]:
#Word2Vec 학습시키기
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model=Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=0)
#vector_size는 데이터가 적을 땐 크게, 데이터가 많을 땐 작게(150~300)
#window는 훈련시 앞 뒤로 고려하는 단어의 개수
#min_count는 해당 빈도수보다 적게 등장할 경우 모델 학습에서 배제
#worker은 스레드 개수. 스레드가 많아질수록 빌드 속도가 빨라진다.(본인 컴퓨터 스펙에 맞게 쓴다.)
#sg 0=CBOW(중심단어 예측) 1=Skipgram(주변단어 예측)

In [24]:
model_result=model.wv.most_similar('man')
#Word2Vec는 입력한 'man'에 대해서 가장 유사한 단어들을 출력해주는 기능이다.
print(model_result)

[('woman', 0.8416481614112854), ('guy', 0.8075767755508423), ('lady', 0.7642030119895935), ('boy', 0.758362352848053), ('girl', 0.7405058145523071), ('gentleman', 0.7271239757537842), ('poet', 0.6821939945220947), ('kid', 0.6817570924758911), ('soldier', 0.6721681356430054), ('friend', 0.6577139496803284)]
